In [6]:
import pandas as pd
import requests
import arrow

In [7]:
#earthquakes = pd.read_csv("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.csv")
#tw_bb = (120.106188593, 21.9705713974, 121.951243931, 25.2954588893)
#taiwan_earthquakes_usgs = earthquakes[(earthquakes["latitude"] >= 21.9705713974) & (earthquakes["latitude"] <= 25.2954588893)]
# Unfortunately, only the major earthquakes made it to usgs' feed
# better source: "http://www.cwb.gov.tw/V7e/earthquake/seismic.htm"

def find_earthquakes(soup):
    earthquakes = []
    for tr in soup.find_all("tr")[1::]:
        content = tr.find_all("a")
        if len(content) > 1:
            ts = arrow.get(content[1].text, tzinfo="+0800")
            lon = float(content[2].text)
            lat = float(content[3].text)
            mag = float(content[4].text)
            desc = content[6].text
            earthquakes.append([ts, lon, lat, mag, desc[1::]])
    return earthquakes

In [8]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import csv

select_months = ["1", "2", "3"]
all_earthquakes = []

# Get earthquake data for Jan, Feb, March
driver = webdriver.PhantomJS()
driver.get("https://scweb.cwb.gov.tw/Page.aspx?ItemId=26&loc=en&gis=n")

for month in select_months:
    select = Select(driver.find_element_by_id("ctl03_ddlMonth"))
    select.select_by_visible_text(month)

    sbtn = driver.find_element_by_id("ctl03_btnSearch")
    sbtn.click()

    soup = BeautifulSoup(driver.page_source, "html5lib")
    all_earthquakes += find_earthquakes(soup)

# cache earthquakes in CSV file
with open("taiwan_earthquakes.csv", "w") as f:
    writer = csv.writer(f)
    for row in all_earthquakes:
        writer.writerow(row)

/Users/joannecheng/dev/notebooks/notebookenv/lib/python3.5/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


In [19]:
import bokeh
from bokeh.plotting import figure, show
from bokeh.models import Range1d, Title, ColumnDataSource, HoverTool

x = list(map(lambda z: z[0].datetime, all_earthquakes))
y = list(map(lambda z: z[3], all_earthquakes))
desc = list(map(lambda z: z[4], all_earthquakes))

ds = ColumnDataSource({
    "x": x,
    "y": y,
    "desc": desc
})

hover = HoverTool(tooltips=[
        ("location", "@desc"),
        ("mag", "@y")
    ])
p = figure(title="Earthquakes in Taiwan, Jan - March 2018", x_axis_type='datetime',
           plot_width=900, plot_height=300, tools=[hover])
p.add_layout(Title(text="source: http://www.cwb.gov.tw/V7e/earthquake/seismic.htm", align="center"), "below")

#p.y_range = Range1d(2,7)

p.yaxis.axis_label = "Magnitude"
p.xaxis.axis_label = "Date"
p.circle("x", "y", source=ds, size=3, color="steelblue", alpha=0.7, )

show(p)

In [ ]:
bokeh.io.reset_output()